In [1]:
#import data

import xlsxwriter
import pandas as pd
from pandas import DataFrame

path = ('/Users/mathijsvanboesschoten/Documents/Efteling/weer_rijen.xlsx')
data = pd.ExcelFile(path)


In [2]:
df1 = data.parse('weer (28:mei)')

# Drop rows with empty cells
df1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df1.head()

,# STN,YYYYMMDD,FG,TG,TX,DR,RH,RHX,RHXH,NG,UG
0,350,19950101,51,18,36,35,35,7,12,5,89
1,350,19950102,51,26,54,39,46,14,6,5,85
2,350,19950103,15,-4,24,0,-1,-1,11,4,89
3,350,19950104,26,-38,3,0,0,0,1,0,93
4,350,19950105,46,-43,-15,0,-1,-1,24,4,74


In [3]:
# add month column 
df1['YYYYMMDD'] = pd.to_datetime(df1['YYYYMMDD'], format='%Y%m%d')
df1['month'] = df1['YYYYMMDD'].dt.month

In [4]:
#change values
df1[['   FG','   TG','   TX','   DR','   RH','  RHX']] = df1[['   FG','   TG','   TX','   DR','   RH','  RHX']]/10

#rename columns
df1 = df1.rename(index=str, columns={"   FG":"avg_windspeed","   TG":"avg_temp","   TX":"max_temp","   DR":"duur_neerslag","   RH":"som_neerslag_mm","  RHX":"hoogste_neerslag_in_uur"," RHXH":"uur_hoogste_neerslag","   NG":"avg_bewolking","   UG":"avg_percent_luchtvochtigheid"})



In [5]:
df1.head()

,# STN,YYYYMMDD,avg_windspeed,avg_temp,max_temp,duur_neerslag,som_neerslag_mm,hoogste_neerslag_in_uur,uur_hoogste_neerslag,avg_bewolking,avg_percent_luchtvochtigheid,month
0,350,1995-01-01,5.1,1.8,3.6,3.5,3.5,0.7,12,5,89,1
1,350,1995-01-02,5.1,2.6,5.4,3.9,4.6,1.4,6,5,85,1
2,350,1995-01-03,1.5,-0.4,2.4,0.0,-0.1,-0.1,11,4,89,1
3,350,1995-01-04,2.6,-3.8,0.3,0.0,0.0,0.0,1,0,93,1
4,350,1995-01-05,4.6,-4.3,-1.5,0.0,-0.1,-0.1,24,4,74,1


Get average weather information per month over the past 20 years

In [6]:
#group by month so we get average weather information over the past 20 years for each month
mean_weather_per_day = df1.groupby("month").mean()
mean_weather_per_day.columns = ['avg_#STN','avg_FG','avg_TG','avg_TX','avg_DR','avg_RH','avg_RHX','avg_RHXH','avg_UG']

In [7]:
mean_weather_per_day.reset_index()

,month,avg_#STN,avg_FG,avg_TG,avg_TX,avg_DR,avg_RH,avg_RHX,avg_RHXH,avg_UG
0,1,350.0,4.247226,3.350194,6.094710,2.388645,2.232387,0.708516,8.574194,87.183226
1,2,350.0,4.200000,4.077337,7.488952,2.547592,2.328187,0.668980,8.719547,84.075071
2,3,350.0,3.980000,6.457806,10.871355,1.836000,1.770452,0.574452,7.085161,79.458065
3,4,350.0,3.617067,9.972000,15.417600,1.201600,1.397867,0.599467,6.833333,73.289333
4,5,350.0,3.567445,13.669390,18.995460,1.484955,2.054604,0.953696,7.267185,73.357977
5,6,350.0,3.319861,16.438472,21.745278,1.161806,2.049722,1.004444,6.705556,74.495833
6,7,350.0,3.209274,18.378091,23.715188,1.299194,2.655780,1.306317,7.533602,75.841398
7,8,350.0,3.048925,17.994220,23.490995,1.260484,2.363710,1.145027,7.094086,77.642473
8,9,350.0,3.140000,14.904167,20.037917,1.331250,2.046389,0.866944,7.104167,81.913889
9,10,350.0,3.565457,11.208468,15.456586,1.713978,2.173118,0.848387,7.538978,85.298387


In [8]:
#add average weather information to existing dataframe
df1_mean_weather = df1.merge(mean_weather_per_day, left_on='month', right_on='month')

In [9]:
#create variables; change in temperature, change in windspeed and change in precipitation
df1_mean_weather['change_maxtemp'] = df1_mean_weather['max_temp'] - df1_mean_weather['avg_TX']
df1_mean_weather['change_avg_temp'] = df1_mean_weather['avg_temp'] - df1_mean_weather['avg_TG']
df1_mean_weather['change_avg_windspeed'] = df1_mean_weather['avg_windspeed'] - df1_mean_weather['avg_TX']
df1_mean_weather['change_duur_neerslag'] = df1_mean_weather['duur_neerslag'] - df1_mean_weather['avg_DR']
df1_mean_weather['change_som_neerslag_mm'] = df1_mean_weather['som_neerslag_mm'] - df1_mean_weather['avg_RH']

# drop average columns per month
df1_mean_weather = df1_mean_weather.drop(columns = ['month','# STN','avg_#STN', 'avg_FG','avg_TG', 'avg_TX', 'avg_DR', 'avg_RH', 'avg_RHX', 'avg_RHXH','avg_UG'])
df1_mean_weather.head()

,YYYYMMDD,avg_windspeed,avg_temp,max_temp,duur_neerslag,som_neerslag_mm,hoogste_neerslag_in_uur,uur_hoogste_neerslag,avg_bewolking,avg_percent_luchtvochtigheid,change_maxtemp,change_avg_temp,change_avg_windspeed,change_duur_neerslag,change_som_neerslag_mm
0,1995-01-01,5.1,1.8,3.6,3.5,3.5,0.7,12,5,89,-2.49471,-1.550194,-0.99471,1.111355,1.267613
1,1995-01-02,5.1,2.6,5.4,3.9,4.6,1.4,6,5,85,-0.69471,-0.750194,-0.99471,1.511355,2.367613
2,1995-01-03,1.5,-0.4,2.4,0.0,-0.1,-0.1,11,4,89,-3.69471,-3.750194,-4.59471,-2.388645,-2.332387
3,1995-01-04,2.6,-3.8,0.3,0.0,0.0,0.0,1,0,93,-5.79471,-7.150194,-3.49471,-2.388645,-2.232387
4,1995-01-05,4.6,-4.3,-1.5,0.0,-0.1,-0.1,24,4,74,-7.59471,-7.650194,-1.49471,-2.388645,-2.332387


## Write to excel

In [10]:
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('data_efteling_change_weather.xlsx', engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object.
# df1_mean_weather.to_excel(writer, sheet_name='data')

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()